# DDPM

In [ ]:
# We will start by generating our data set: a tensor of 1000 samples following a multivariate normal distribution with a random mean and a random covariance matrix. We will use the torch.distributions module to generate the data set. The code is as follows:

In [1]:
import torch
import torch.distributions as dist
import matplotlib.pyplot as plt

In [3]:
X = dist.MultivariateNormal(torch.rand(2), torch.tensor([[0.5, 0.2], [0.2, 0.5]])).sample((1000,))

In [4]:
# Starting with model hyperparameters

steps = 1000
max_epoch = 10

beta_1 = 1e-4
beta_T = 1e-2

tensor([[0.4332, 1.0367],
        [1.6569, 0.7842],
        [0.2922, 0.5216],
        ...,
        [0.6477, 0.7486],
        [0.8193, 0.5063],
        [1.2184, 1.0791]])